In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
import math

from trackml.dataset import load_event, load_dataset
from trackml.score import score_event

from multiprocessing import Pool

from itertools import product

def create_one_event_submission(event_id, hits, labels):
    sub_data = np.column_stack(([event_id]*len(hits), hits.hit_id.values, labels))
    submission = pd.DataFrame(data=sub_data, columns=["event_id", "hit_id", "track_id"]).astype(int)
    return submission

In [2]:
# Change this according to your directory preferred setting
path_to_train = "./data/train_1"

# This event is in Train_1
event_prefix = "event000001000"

In [3]:
weights_0 = np.array([1.1, 1.1, .375, .25, 0.05, 0.05, 0])
weights_2 = np.array([1.05, 1.05, .37, .25, .01, .045, 0])
weights_3 = np.array([1.08, 1.08, .38,.25, 0.009, 0.0001, 0])
weights_4 = np.array([1.1, 1.1, .5, .25, 0.008, 0.001, 0])
weights_5 = np.array([1.01, 1.01, .4048, .2, 2e-16, 2e-16, 0])
weights_9 = np.array([1.02, 1.02, .38, .25, 0.009, 0.02, 0])

weights_1 = np.array([1.08, 1.08, .38,.25, 0.0001, 0.0001, 0])
weights_6 = np.array([1.08, 1.08, .38, .25, 2e-4, 0, 0])
weights_8 = np.array([1.07, 1.07, .37, .25, 0.0002, 0.0001, 0])
weights_12 = np.array([1.01, 1.01, .42, .25, 0.0023, 0.001, 0])
weights_13 = np.array([1.0, 1.0, .40, .20, 0.0023, 0.0001, 0])
weights_14 = np.array([1.02, 1.02, .40, .24, 0.0002, 0, 0])
weights_7 = np.array([1.08, 1.08, .40, .20, 0.0023, 0, 0])

weights_10 = np.array([1.02, 1.02, .42, .24, 0.0002, 0, 0])
weights_11 = np.array([1.08, 1.08, .40, .20, 0.0023, 0.0001, 0])
weights_15 = np.array([1.01, 1.01, .42, .25, 0.0002, 0, 0])
weights_16 = np.array([1.005, 1.005, .40, .25, 0.0002, 0, 0])
weights_17 = np.array([1.02, 1.02, .42, .24, 0.02, 0, 0])
weights_18 = np.array([1.033, 1.033, .376, .2, 0.0002, 0, 0])
weights_19 = np.array([0.9734, 0.9734, .4072, .2217, 0.0, 0.0, 0])
weights_20 = np.array([1.0, 1.0, .42, .24, 0.0, 0, 0])
weights_21 = np.array([1.02, 1.02, .42, .24, 0.0, 0, 0])
weights_22 = np.array([1.05, 1.05, .4, .25, 0.0, 0, 0])
weights_23 = np.array([1.05, 1.05, .5, .25, 0.0, 0, 0])
weights_24 = np.array([1.05, 1.05, .4, .20, 0.0, 0, 0])
weights_26 = np.array([1.2, 1.2, .45, .15, 0.0, 0, 0])
weights_27 = np.array([1.2, 1.2, .45, .20, 0.0, 0, 0])
weights_28 = np.array([1.2, 1.2, .45, .25, 0.0, 0, 0])
weights_29 = np.array([1.09, 1.09, .5, .20, 0.0, 0, 0])
weights_25 = np.array([1.09, 1.09, .45, .20, 0.0, 0, 0])
weights_30 = np.array([0.9, .9, .35, .15, 0.0, 0.007, 0.007])
weights_31 = np.array([1.09, 1.09, .45, .20, 0.0, 0.005, 0.005])
weights_34 = np.array([1.09, 1.09, .45, .20, 0.0, 0.01, 0.01])
weights_36 = np.array([1.0, 1.0, .35, .2, 0.0, 0.02, 0.02])
weights_32 = np.array([0.9, .9, .35, .2, 0.0, 0.01, 0.01])
weights_33 = np.array([1.0, 1.0, .35, .2, 0.0, 0.01, 0.01])
weights_37 = np.array([0.9, .9, .4, .2, 0.0, 0.015, 0.015]) # raise w4
weights_38 = np.array([0.9, .9, .4, .25, 0.0, 0.01, 0.01])  # raise w3
weights_39 = np.array([0.9, .9, .45, .2, 0.0, 0.01, 0.01])
weights_40 = np.array([0.95, .95, .4, .2, 0.0, 0.01, 0.01])
weights_41 = np.array([0.95, .95, .45, .2, 0.0, 0.01, 0.01])  # 38 + 40
weights_42 = np.array([0.95, .95, .4, .2, 0.0, 0.008, 0.008]) # w4 lowered
weights_43 = np.array([1.0, 1.0, .4, .2, 0.0, 0.01, 0.01])    # w1 raised
weights_44 = np.array([0.95, .95, .4, .2, 0.0, 0.012, 0.012]) # w4 raised
weights_45 = np.array([1.0, 1.0, .4, .2, 0.0, 0.01, 0.01])  # raise w1 from 35
weights_46 = np.array([0.9, .9, .4, .2, 0.0, 0.009, 0.009]) # lower w4
weights_47 = np.array([0.85, .85, .4, .2, 0.0, 0.01, 0.01]) # lower w1
weights_49 = np.array([0.9, .9, .4, .2, 0.0, 0.007, 0.007]) # still lower w4
weights_48 = np.array([0.9, .9, .4, .2, 0.0, 0.008, 0.008]) # lower w4
weights_51 = np.array([0.9, .9, .4, .2, 0.0, 0.009, 0.009]) # lower w4
weights_50 = np.array([0.9, .9, .4, .2, 0.0, 0.011, 0.011]) # higher w4
weights_52 = np.array([0.9, .9, .4, .2, 0.0, 0.012, 0.012]) # higher w4
weights_53 = np.array([0.9, .9, .4, .2, 0.0, 0.013, 0.013]) # higher w4
weights_54 = np.array([0.9, .9, .4, .2, 0.0, 0.014, 0.014]) # higher w4
weights_55 = np.array([0.9, .9, .4, .2, 0.0, 0.015, 0.015]) # higher w4
weights_56 = np.array([0.9, .9, .5, .2, 0.0, 0.01, 0.01])     # raise w2
weights_60 = np.array([0.7, .7, .5, .13, 0.0, 0.01, 0.01])     # raise w2, lower w1, w3
weights_59 = np.array([0.9, .9, .55, .1, 0.0, 0.01, 0.01])     # raise w2, lower w3
weights_35 = np.array([0.9, .9, .4, .2, 0.0, 0.01, 0.01])     
weights_57 = np.array([0.9, .9, .5, .15, 0.0, 0.01, 0.01]) # second best
weights_61 = np.array([0.9, .9, .4, .1, 0.0, 0.01, 0.01])  # lower w3 some more
weights_62 = np.array([0.9, .9, .45, .1, 0.0, 0.01, 0.01])  # lower w3 and raise w2
weights_63 = np.array([0.9, .9, .5, .1, 0.0, 0.01, 0.01])  # raise w2 some more
weights_64 = np.array([0.9, .9, .5, .07, 0.0, 0.01, 0.01])  # raise w2 and lower w3 some more
weights_65 = np.array([0.9, .9, .35, .15, 0.0, 0.01, 0.01])   # lower w2 a bit
weights_66 = np.array([0.9, .9, .35, .2, 0.0, 0.01, 0.01])   # lower w2 a bit and raise w3
weights_67 = np.array([0.95, .95, .3, .2, 0.0, 0.012, 0.012])# weights from r optimization
weights_68 = np.array([0.96, .96, .35, .2, 0.0, 0.0085, 0.0085])# weights from r optimization
weights_69 = np.array([0.9, .9, .4, .15, 0.0, 0.0085, 0.0085])   # w4 down

weights_58 = np.array([0.9, .9, .4, .15, 0.0, 0.01, 0.01])   # best so far
weights_70 = np.array([1.0, 1.0, .4, .1214, 0.0, 0.008, 0.008])   # w4 down
weights_71 = np.array([0.9, 0.9, .5, .15, 0.0, 0.0085, 0.0085])   # w4 down

weights_arr = np.vstack([weights_0, weights_1, weights_2, weights_3, weights_4, weights_5, weights_6, weights_7, weights_8, weights_9, weights_10, weights_11, weights_12, weights_13, weights_14,  weights_15,  weights_16,  weights_17,  weights_18,  weights_19,  weights_20,  weights_21,  weights_22,  weights_23,  weights_24,  weights_25,  weights_26,  weights_27,  weights_28,  weights_29,  weights_30,  weights_31,  weights_32,  weights_33,  weights_34,  weights_35,  weights_36,  weights_37,  weights_38,  weights_39,  weights_40,  weights_41,  weights_42,  weights_43,  weights_44,  weights_45,  weights_46,  weights_47,  weights_48,  weights_49,  weights_50,  weights_51,  weights_52,  weights_53,  weights_54,  weights_55,  weights_56,  weights_57,  weights_58,  weights_59,  weights_60,  weights_61,  weights_62,  weights_63,  weights_64,  weights_65,  weights_66,  weights_67,  weights_68,  weights_69,  weights_70,  weights_71])

In [4]:
from sklearn.preprocessing import StandardScaler
import hdbscan
from scipy import stats
from tqdm import tqdm_notebook as tqdm
from sklearn.cluster import DBSCAN

class Clusterer(object):
    def __init__(self,rz_scales=[0.65, 0.965, 1.5, 1.5], eps=0.0035, dz0 = -0.0007, stepdz = 1.5e-7, stepeps = 1.5e-09, num_loops=175, final_cluster=True, weight=58, weight_arr=weights_arr, max_size=20, step_z=0.000045, size_incr=0):
        self.rz_scales=rz_scales
        self.epsilon = eps
        self.dz0 = dz0
        self.stepdz = stepdz
        self.stepeps = stepeps
        self.num_loops = num_loops
        self.final_cluster = final_cluster
        self.weight_arr = weight_arr
        self.weights = weight
        self.max_size = max_size
        self.step_z = step_z
        self.size_incr = size_incr
        self.min_points = 1
        
    # remove outliers
    def _eliminate_outliers(self,labels,M):
        norms=np.zeros((len(labels)),np.float32)
        indices=np.zeros((len(labels)),np.float32)
        for i, cluster in tqdm(enumerate(labels),total=len(labels)):
            if cluster == 0:
                continue
            index = np.argwhere(self.clusters==cluster)
            index = np.reshape(index,(index.shape[0]))
            indices[i] = len(index)
            x = M[index]
            norms[i] = self._test_quadric(x)
        threshold1 = np.percentile(norms,90)*5
        threshold2 = 25
        threshold3 = 5
        for i, cluster in enumerate(labels):
            if norms[i] > threshold1 or indices[i] > threshold2 or indices[i] < threshold3:
                self.clusters[self.clusters==cluster]=0   
    
    # not sure what this function does?
    def _test_quadric(self,x):
        if x.size == 0 or len(x.shape)<2:
            return 0
        Z = np.zeros((x.shape[0],10), np.float32)
        Z[:,0] = x[:,0]**2
        Z[:,1] = 2*x[:,0]*x[:,1]
        Z[:,2] = 2*x[:,0]*x[:,2]
        Z[:,3] = 2*x[:,0]
        Z[:,4] = x[:,1]**2
        Z[:,5] = 2*x[:,1]*x[:,2]
        Z[:,6] = 2*x[:,1]
        Z[:,7] = x[:,2]**2
        Z[:,8] = 2*x[:,2]
        Z[:,9] = 1
        v, s, t = np.linalg.svd(Z,full_matrices=False)        
        smallest_index = np.argmin(np.array(s))
        T = np.array(t)
        T = T[smallest_index,:]        
        norm = np.linalg.norm(np.dot(Z,T), ord=2)**2
        return norm

    # standard scale our data
    def _preprocess(self, hits):
        x = hits.x.values
        y = hits.y.values
        z = hits.z.values

        r = np.sqrt(x**2 + y**2 + z**2)
        rt = np.sqrt(x**2 + y**2)
        
        hits['a0'] = np.arctan2(hits['y'].values,hits['x'].values)
        hits['a1'] = hits['a0']+1*(hits['rt']+1*hits['rt']**2)/1000*(1/2)/180*math.pi
        
        hits['x2'] = x/r
        hits['y2'] = y/r
        hits['z1'] = z/rt
        hits['z2'] = z/r

        ss = StandardScaler()
        X = ss.fit_transform(hits[['sina1','cosa1','z1','z2', 'z3', 'x2', 'y2']].values)
        X = np.multiply(X, self.weight_arr[self.weights])
       
        return X
        
        # original preprocess
#         x = hits.x.values
#         y = hits.y.values
#         z = hits.z.values

#         r = np.sqrt(x**2 + y**2 + z**2)
#         hits['x2'] = x/r
#         hits['y2'] = y/r

#         rt = np.sqrt(x**2 + y**2)
#         hits['z1'] = z/rt
#         hits['z2'] = z/r

#         ss = StandardScaler()
#         X = ss.fit_transform(hits[['x2', 'y2', 'z1', 'z2']].values)
#         for i, rz_scale in enumerate(self.rz_scales):
#             X[:,i] = X[:,i] * rz_scale
       
#         return X
    
    def _init(self,dfh):
        dfh['s1'] = dfh.hit_id
        dfh['N1'] =1
        dfh['stepped_z'] = dfh.z
        mm = 1
        dz0 = self.dz0
        min_points = self.min_points
        
        for ii in range(self.num_loops):
            max_size = self.max_size + (ii * self.size_incr)
            
            dfh['r'] = np.sqrt(dfh['x'].values**2+dfh['y'].values**2+dfh['stepped_z'].values**2)
            dfh['rt'] = np.sqrt(dfh['x'].values**2+dfh['y'].values**2)
            dfh['a0'] = np.arctan2(dfh['y'].values,dfh['x'].values)
            dfh['z1'] = dfh['stepped_z'].values/dfh['rt'].values
            
            dfh['z2'] = dfh['stepped_z']/dfh['r']
            dfh['x2'] = dfh['x'].values/dfh['r'].values
            dfh['y2'] = dfh['y'].values/dfh['r'].values
            
            dfh['z3'] = 0 # 1/dfh['z1'].values - this feature didn't work, let's 0 it out
            mm = mm*(-1)  
            
            z_step = mm * self.step_z * ii
            dz = mm*(self.dz0+(ii*self.stepdz))
            
            dfh['stepped_z'] = dfh['z'] + z_step    
            
            dfh['a1'] = dfh['a0']+mm*(dfh['rt']+(dz)*dfh['rt']**2)/1000*(ii/2)/180*math.pi
            dfh['sina1'] = np.sin(dfh['a1'].values)
            dfh['cosa1'] = np.cos(dfh['a1'].values)
            ss = StandardScaler()
            dfs = ss.fit_transform(dfh[['sina1','cosa1','z1','z2', 'z3', 'x2', 'y2']].values)
            cx = self.weight_arr[self.weights]
            dfs = np.multiply(dfs, cx)

            clusters=DBSCAN(eps=self.epsilon+(ii*self.stepeps),min_samples=min_points,metric='euclidean',n_jobs=1).fit(dfs).labels_ 
            
            # optionally change min points
            min_points = min_points + (ii + self.min_pts_change)
            
            if ii==0:
                dfh['s1'] = clusters
                dfh['N1'] = dfh.groupby('s1')['s1'].transform('count')

            # else update our hits conditionally, if it's a better fit
            else:
                # put our new clusters to another feature
                dfh['s2'] = clusters

                # get the count of those clusters
                dfh['N2'] = dfh.groupby('s2')['s2'].transform('count')
                maxs1 = dfh['s1'].max()

                # if our new clusters are bigger, but less than our max size, use the new ones instead
                cond = np.where((dfh['N2'].values > dfh['N1'].values) & (dfh['N2'].values < max_size))
                    
                s1 = dfh['s1'].values
                s1[cond] = dfh['s2'].values[cond]+maxs1

                # write the new clusters back to our dataframe
                dfh['s1'] = s1
                dfh['s1'] = dfh['s1'].astype('int64')
                dfh['N1'] = dfh.groupby('s1')['s1'].transform('count')
        
        # for debugging
        # return dfh
        
        # return our clusters
        return dfh['s1'].values    
    
    def predict(self, hits):    
        # init our clusters
        self.clusters = self._init(hits) 
        
        mask = self.clusters == 0
        
        if self.final_cluster:
            # preprocess our data
            X = self._preprocess(hits) 

            # create our last clusterer
            cl = hdbscan.HDBSCAN(min_samples=1, min_cluster_size=7, metric='braycurtis', cluster_selection_method='leaf', algorithm='best', leaf_size=50)

            # labels = unique clusters
            labels = np.unique(self.clusters)

            # remove outliers
            self._eliminate_outliers(labels,X)

            # init n_labels
            n_labels = 0

            # now we loop through the points that haven't been assigned to a cluster and assign them with
            # HDBSCAN
            while n_labels < len(labels):
                n_labels = len(labels)
                max_len = np.max(self.clusters)
                mask = self.clusters == 0
            self.clusters[mask] = cl.fit_predict(X[mask])+max_len
            
        return self.clusters

In [5]:
results_df = pd.read_pickle("gs_results.pkl")
# results_df

In [6]:
# defaults: 
# eps=0.0035, dz0 = -0.00070, stepdz = 0.00001, stepeps = 0.000005
# 0.0035, 0.0005, 5e-05, 1e-09, 150, True, 10, True
# create our params to iterate over
eps_vals = [ 0.0035 ] # try lowering with increased step eps
stepdz_vals = [ 1.5e-7] 
stepeps_vals = [ 5e-7, 7e-8 ] # 1.5e-9, 
loop_vals = [ 250 ]
weight_vals = [ 70, 58 ]
step_z_vals = [ 4.5e-5, 3.5e-5 ]

size_vals = [ 20 ]

dz0_vals = [ -0.00070 ] # unused
final_cluster = [ True ] # unused
dz_incr = [ True ] # unused
size_inc_vals = [ 0 ]
step = "eps"

foo = product(eps_vals, dz0_vals, stepdz_vals, stepeps_vals, loop_vals, final_cluster, weight_vals, dz_incr, size_vals, step_z_vals, size_inc_vals)

offset = np.max(results_df.index.values) + 1

iter_list = []
for i, item in enumerate(foo):
    iter_list.append([i + offset, item])
    
print("Length:", len(iter_list))    

Length: 8


In [7]:
def grid_search_loop(params):
    counter = params[0]
    eps, dz0, stepdz, stepeps, loops, final_cluster, weights, dz_incr, max_size, step_z, size_incr = params[1]
    step = "eps"
    dataset_scores = []
    print(" - ", counter, "Eps:", eps, "stepdz:", stepdz, "stepeps:", stepeps, "loops:", loops, "weight:", weights, "max size:", max_size, "step z:", step_z)
    
    for event_id, hits, cells, particles, truth in load_dataset("./data/train_1", skip=40, nevents=10):
        # Track pattern recognition
        model = Clusterer(eps=eps, dz0=dz0, stepdz=stepdz, stepeps=stepeps, num_loops=loops, final_cluster=final_cluster, weight=weights, weight_arr=weights_arr, max_size=max_size, step_z=step_z, size_incr=size_incr)
        labels = model.predict(hits)

        # Prepare submission for an event
        one_submission = create_one_event_submission(event_id, hits, labels)

        # Score for the event
        score = score_event(truth, one_submission)
        dataset_scores.append(score)
        print(event_id, params[1], "score:", score)
        
    mean_score = np.mean(dataset_scores)
    
    # create and return our results
    result = [counter, [eps, dz0, stepdz, stepeps, loops, step, weights, final_cluster, max_size, step_z, size_incr, mean_score]]
    print(result)
    
    return result

def grid_search(iter_list, results_df=None, start=0, end=5):
    pool = Pool(processes=14)
    results = pool.map(grid_search_loop, iter_list[start:end])
    pool.close()
    
    return results

In [8]:
results = grid_search(iter_list, results_df, start=0, end=28)

for item in results:
    results_df.loc[item[0]] = item[1]
    
results_df.sort_values("acc", ascending=False).to_pickle("gs_results.pkl")
results_df

 -  1192 Eps: 0.0035 stepdz: 1.5e-07 stepeps: 5e-07 loops: 250 weight: 70 max size: 20 step z: 3.5e-05
 -  1191 Eps: 0.0035 stepdz: 1.5e-07 stepeps: 5e-07 loops: 250 weight: 70 max size: 20 step z: 4.5e-05
 -  1193 Eps: 0.0035 stepdz: 1.5e-07 stepeps: 5e-07 loops: 250 weight: 58 max size: 20 step z: 4.5e-05
 -  1195 Eps: 0.0035 stepdz: 1.5e-07 stepeps: 7e-08 loops: 250 weight: 70 max size: 20 step z: 4.5e-05
 -  1194 Eps: 0.0035 stepdz: 1.5e-07 stepeps: 5e-07 loops: 250 weight: 58 max size: 20 step z: 3.5e-05
 -  1196 Eps: 0.0035 stepdz: 1.5e-07 stepeps: 7e-08 loops: 250 weight: 70 max size: 20 step z: 3.5e-05
 -  1197 Eps: 0.0035 stepdz: 1.5e-07 stepeps: 7e-08 loops: 250 weight: 58 max size: 20 step z: 4.5e-05
 -  1198 Eps: 0.0035 stepdz: 1.5e-07 stepeps: 7e-08 loops: 250 weight: 58 max size: 20 step z: 3.5e-05


/home/skooch/.local/lib/python3.5/site-packages/sklearn/externals/joblib/parallel.py:547: UserWarning: Multiprocessing-backed parallel loops cannot be nested, setting n_jobs=1
  **self._backend_args)
/home/skooch/.local/lib/python3.5/site-packages/sklearn/externals/joblib/parallel.py:547: UserWarning: Multiprocessing-backed parallel loops cannot be nested, setting n_jobs=1
  **self._backend_args)
/home/skooch/.local/lib/python3.5/site-packages/sklearn/externals/joblib/parallel.py:547: UserWarning: Multiprocessing-backed parallel loops cannot be nested, setting n_jobs=1
  **self._backend_args)
/home/skooch/.local/lib/python3.5/site-packages/sklearn/externals/joblib/parallel.py:547: UserWarning: Multiprocessing-backed parallel loops cannot be nested, setting n_jobs=1
  **self._backend_args)


/home/skooch/.local/lib/python3.5/site-packages/sklearn/externals/joblib/parallel.py:547: UserWarning: Multiprocessing-backed parallel loops cannot be nested, setting n_jobs=1
  **self._backend_args)
/home/skooch/.local/lib/python3.5/site-packages/sklearn/externals/joblib/parallel.py:547: UserWarning: Multiprocessing-backed parallel loops cannot be nested, setting n_jobs=1
  **self._backend_args)
/home/skooch/.local/lib/python3.5/site-packages/sklearn/externals/joblib/parallel.py:547: UserWarning: Multiprocessing-backed parallel loops cannot be nested, setting n_jobs=1
  **self._backend_args)
/home/skooch/.local/lib/python3.5/site-packages/sklearn/externals/joblib/parallel.py:547: UserWarning: Multiprocessing-backed parallel loops cannot be nested, setting n_jobs=1
  **self._backend_args)


1040 (0.0035, -0.0007, 1.5e-07, 7e-08, 250, True, 70, True, 20, 3.5e-05, 0) score: 0.5031343156056209
1040 (0.0035, -0.0007, 1.5e-07, 7e-08, 250, True, 70, True, 20, 4.5e-05, 0) score: 0.5031343156056209
1040 (0.0035, -0.0007, 1.5e-07, 7e-08, 250, True, 58, True, 20, 4.5e-05, 0) score: 0.5034504303273479
1040 (0.0035, -0.0007, 1.5e-07, 7e-08, 250, True, 58, True, 20, 3.5e-05, 0) score: 0.5034504303273479
1040 (0.0035, -0.0007, 1.5e-07, 5e-07, 250, True, 58, True, 20, 3.5e-05, 0) score: 0.5031772259505942
1040 (0.0035, -0.0007, 1.5e-07, 5e-07, 250, True, 70, True, 20, 3.5e-05, 0) score: 0.5026417827932619
1040 (0.0035, -0.0007, 1.5e-07, 5e-07, 250, True, 70, True, 20, 4.5e-05, 0) score: 0.5026329746935971
1040 (0.0035, -0.0007, 1.5e-07, 5e-07, 250, True, 58, True, 20, 4.5e-05, 0) score: 0.5031772259505942


/home/skooch/.local/lib/python3.5/site-packages/sklearn/externals/joblib/parallel.py:547: UserWarning: Multiprocessing-backed parallel loops cannot be nested, setting n_jobs=1
  **self._backend_args)
/home/skooch/.local/lib/python3.5/site-packages/sklearn/externals/joblib/parallel.py:547: UserWarning: Multiprocessing-backed parallel loops cannot be nested, setting n_jobs=1
  **self._backend_args)


...

/home/skooch/.local/lib/python3.5/site-packages/sklearn/externals/joblib/parallel.py:547: UserWarning: Multiprocessing-backed parallel loops cannot be nested, setting n_jobs=1
  **self._backend_args)


1049 (0.0035, -0.0007, 1.5e-07, 7e-08, 250, True, 70, True, 20, 3.5e-05, 0) score: 0.5145118571981584
[1196, [0.0035, -0.0007, 1.5e-07, 7e-08, 250, 'eps', 70, True, 20, 3.5e-05, 0, 0.5214383695241115]]
1049 (0.0035, -0.0007, 1.5e-07, 5e-07, 250, True, 70, True, 20, 3.5e-05, 0) score: 0.5144711861626092
[1192, [0.0035, -0.0007, 1.5e-07, 5e-07, 250, 'eps', 70, True, 20, 3.5e-05, 0, 0.5221316521364199]]


/home/skooch/.local/lib/python3.5/site-packages/sklearn/externals/joblib/parallel.py:547: UserWarning: Multiprocessing-backed parallel loops cannot be nested, setting n_jobs=1
  **self._backend_args)


1049 (0.0035, -0.0007, 1.5e-07, 5e-07, 250, True, 70, True, 20, 4.5e-05, 0) score: 0.5145517405594546
[1191, [0.0035, -0.0007, 1.5e-07, 5e-07, 250, 'eps', 70, True, 20, 4.5e-05, 0, 0.5221521707821181]]


,eps,dz0,stepdz,stepeps,loops,step,weights,final_cluster,max_size,step_z,size_incr,acc
1137,0.00350,-0.0007,1.500000e-07,7.000000e-08,250.0,eps,58,True,20,0.000045,0.00,0.530545
1143,0.00350,-0.0007,1.500000e-07,1.500000e-09,250.0,eps,58,True,20,0.000045,0.00,0.530490
1138,0.00350,-0.0007,1.500000e-07,7.000000e-08,250.0,eps,58,True,20,0.000002,0.00,0.530473
1128,0.00350,-0.0007,1.500000e-07,1.500000e-09,250.0,eps,58,True,20,0.000002,0.00,0.530456
1118,0.00350,-0.0007,1.500000e-07,1.500000e-09,200.0,eps,58,True,20,0.000002,0.00,0.530382
1117,0.00350,-0.0007,1.500000e-07,1.500000e-09,200.0,eps,58,True,20,0.000000,0.00,0.530377
1133,0.00350,-0.0007,1.500000e-07,1.500000e-09,275.0,eps,58,True,20,0.000002,0.00,0.530093
1123,0.00350,-0.0007,1.500000e-07,1.500000e-09,200.0,eps,58,True,20,0.000002,0.00,0.529915
1131,0.00350,-0.0007,1.500000e-07,1.500000e-09,250.0,eps,65,True,20,0.000002,0.00,0.529439
1121,0.00350,-0.0007,1.500000e-07,1.500000e-09,200.0,eps,62,True,20,0.000000,0.00,0.529137


In [9]:
results_df.sort_values("acc", ascending=False)

,eps,dz0,stepdz,stepeps,loops,step,weights,final_cluster,max_size,step_z,size_incr,acc
1099,0.00350,-0.0007,1.500000e-07,1.500000e-09,175.0,eps,35,True,20,0.000000,0.00,0.524089
1100,0.00350,-0.0007,1.500000e-07,1.500000e-09,175.0,eps,35,True,20,0.000002,0.00,0.524082
1088,0.00350,-0.0007,1.500000e-07,1.500000e-09,175.0,eps,50,True,20,0.000002,0.00,0.523875
1087,0.00350,-0.0007,1.500000e-07,1.500000e-09,175.0,eps,50,True,20,0.000000,0.00,0.523870
1089,0.00350,-0.0007,1.500000e-07,1.500000e-09,175.0,eps,51,True,20,0.000000,0.00,0.523767
1090,0.00350,-0.0007,1.500000e-07,1.500000e-09,175.0,eps,51,True,20,0.000002,0.00,0.523767
1092,0.00350,-0.0007,1.500000e-07,1.500000e-09,175.0,eps,52,True,20,0.000002,0.00,0.523473
1091,0.00350,-0.0007,1.500000e-07,1.500000e-09,175.0,eps,52,True,20,0.000000,0.00,0.523473
1085,0.00350,-0.0007,1.500000e-07,1.500000e-09,175.0,eps,35,True,20,0.000000,0.00,0.523135
1086,0.00350,-0.0007,1.500000e-07,1.500000e-09,175.0,eps,35,True,20,0.000002,0.00,0.523126


In [12]:
results_df.sort_values("acc", ascending=False).to_pickle("gs_results.pkl")